### This script contains the following points:

#### 1. Importing libraries
#### 2. Importing product and order data
#### 3. Describing order data
#### 4. Detecting mixed data types in order data
#### 5. Detecting and addressing missing values in order data
#### 6. Detecting duplicates in order data
#### 7. Removing missing values and duplicates from products data
#### 8. Exporting data

# 1. Importing libraries

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

# 2. Importing data

In [3]:
# importing products from Original data
df_prods = pd.read_csv(r'C:\Users\kevan\Documents\Career Foundry\Data Immersion\Achievement 4\Instacart Basket Analysis\02 Data\Original Data\products.csv', index_col = False)

In [4]:
# importing orders dataframe
# index_col = [0] to remove default index column in df
df_ords = pd.read_csv(r'C:\Users\kevan\Documents\Career Foundry\Data Immersion\Achievement 4\Instacart Basket Analysis\02 Data\Prepared Data\orders_wrangled.csv', index_col = [0])

In [4]:
# assigning main project path to variable 'path'
path = r'C:\Users\kevan\Documents\Career Foundry\Data Immersion\Achievement 4\Instacart Basket Analysis'

In [5]:
path

'C:\\Users\\kevan\\Documents\\Career Foundry\\Data Immersion\\Achievement 4\\Instacart Basket Analysis'

# 3. Describing order data

#### Count of all values (except days since prior order): 3.421083e+06
#### Order ID min: 1, max: 3.421083e+06
#### Max order ID matches total counts, but order IDs are not listed in sequential order. How are order IDs assigned to each user?
#### User order number max: 100. Did someone actually order 100 times?

In [6]:
# descriptive statistics for order data

df_ords.describe()

,order_id,user_id,user_order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


# 4. Detecting mixed data types in order data

In [7]:
# Check for mixed types in order columns
# None detected

for col in df_ords.columns.tolist():
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords[weird]) > 0:
    print (col)

# 5. Detecting and addressing missing values in order data

In [9]:
# Finding missing values in orders dataframe

df_ords.isnull().sum()

order_id                       0
user_id                        0
user_order_number              0
orders_day_of_week             0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

In [10]:
# subset of orders with missing values

df_nan_ords = df_ords[df_ords['days_since_prior_order'].isnull() == True]
df_nan_ords

,order_id,user_id,user_order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
11,2168274,2,1,2,11,NaN
26,1374495,3,1,1,14,NaN
39,3343014,4,1,6,11,NaN
45,2717275,5,1,3,12,NaN
...,...,...,...,...,...,...
3420930,969311,206205,1,4,12,NaN
3420934,3189322,206206,1,3,18,NaN
3421002,2166133,206207,1,6,19,NaN
3421019,2227043,206208,1,1,15,NaN


### The only column with missing values is days_since_prior_order. This may be because a lot of users have only used the app once, so a prior order doesn't exist.

In [11]:
# shape of orders dataframe

df_ords.shape

(3421083, 6)

### Kept null values because this is the initial order for each user. Replaced null with N/A to flag as initial order

In [4]:
# replace nulls with N/A 

df_ords['days_since_prior_order'].fillna("N/A", inplace = True)
df_ords

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,N/A
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0
...,...,...,...,...,...,...
3421078,2266710,206209,10,5,18,29.0
3421079,1854736,206209,11,4,10,30.0
3421080,626363,206209,12,1,12,18.0
3421081,2977660,206209,13,1,12,7.0


# 6. Detecting duplicates in order data

In [13]:
# searching for duplicates in orders df
# none found

df_ords[df_ords.duplicated()]

,order_id,user_id,user_order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order


In [15]:
# another way to see a snapshot of data to see if duplicate values are present

df_ords.duplicated().value_counts()

False    3421083
dtype: int64

# 7. Removing missing values and duplicates from products data

In [5]:
# removing missing values and creating new dataframe

df_prods_clean = df_prods[df_prods['product_name'].isnull() == False]

In [6]:
df_prods_clean_no_dups = df_prods_clean.drop_duplicates()

# 8. Exporting data

In [8]:
# exporting clean products data frame

df_prods_clean_no_dups.to_csv(os.path.join(path, '02 Data','Prepared Data', 'products_checked.csv'))

In [9]:
# exporting clean order data frame

df_ords.to_csv(os.path.join(path, '02 Data','Prepared Data', 'orders_checked.csv'))